In [10]:
%reload_ext autoreload
%autoreload 2

# Generate the RepCode circuit

In [11]:
from result_saver import SaverProvider

provider = SaverProvider()

In [194]:
from qiskit_qec.circuits import RepetitionCodeCircuit
from soft_info import get_repcode_layout

DISTANCE = 11
ROUNDS = 11
DEVICE = "ibmq_mumbai"
LOGICAL = '0'
SHOTS = 1e4

code = RepetitionCodeCircuit(DISTANCE, ROUNDS)
backend = provider.get_backend(DEVICE)
layout = get_repcode_layout(DISTANCE, backend, _is_hex=False)

qc = code.circuit[LOGICAL]                                   

Finding the longest path starting from 27 qubits: 100%|██████████| 27/27 [00:00<00:00, 38956.38it/s]


# Noise model

In [195]:
from qiskit_qec.noise import PauliNoiseModel

def get_noise_model(p1Q, p2Q, pXY, pZ, pRO, pRE):

    error_dict = {
        'reset': {
            "chan": {
                        'i':1-pRE,
                        'x':pRE
                    }
                },
        'measure': {
            "chan": {
                        'i':1-pRO,
                        'x':pRO
                    }
                },
        'h': {
            "chan": {
                        'i':1-p1Q
                    }|
                    {
                        i:p1Q/3
                        for i in 'xyz'
                    }
                },
        #idling error attached to a custom gate "idle_1" (assumed to be the identity acting on a single qubit)
        'idle_1': {
            "chan": {
                        'i':1-pXY,
                        'x':pXY/2,
                        'y':pXY/2
                    }
                },
        #another type of idling error attached to a custom gate "idle_2" (assumed to be the identity acting on a single qubit)
        'idle_2': {
            "chan": {
                        'i':1-pZ,
                        'z':pZ
                    }
                },

        'cx': {
            "chan": {
                        'ii':1-p2Q
                    }|
                    {
                        i+j:p2Q/15
                        for i in 'ixyz'
                        for j in 'ixyz'
                        if i+j!='ii'
                    }
                },
        'swap': {
            "chan": {
                        'ii':1-p2Q
                    }|
                    {
                        i+j:p2Q/15
                        for i in 'ixyz'
                        for j in 'ixyz'
                        if i+j!='ii'
                    }
                }
                }

    noise_model = PauliNoiseModel(fromdict=error_dict)

    return noise_model



# Get counts via stim

In [196]:
from qiskit_qec.utils import get_counts_via_stim

p1Q = 1e-4
p2Q = 6e-3
pXY = 1e-4
pZ = 1e-4
pRO = 1e-1
pRE = 0

noise_model = get_noise_model(p1Q, p2Q, pXY, pZ, pRO, pRE)
needed_nb_samples = get_counts_via_stim(qc, shots=int(SHOTS), noise_model=noise_model)
print(type(needed_nb_samples))
sorted_counts = sorted(needed_nb_samples.items(), key=lambda x: x[1], reverse=True)
# (sorted_counts)

<class 'dict'>


[('00000000000 0101110111 0101110110 0101110110 0101110010 0101110010 0100100010 0000000010 0000000010 0000000000 0000000000 0000000000',
  1),
 ('01001000000 1000011101 0111111101 1010011101 0111100101 0010100101 1110100101 0000100001 1100100001 0000100001 0000000001 0000000000',
  1),
 ('00000010101 1110101000 1110100000 1110100000 0010100000 0010100000 0010000000 1000000000 1000000000 1000000000 1000010000 0000010000',
  1),
 ('00010010000 1001011100 1001011100 1001011100 1001011100 0001010101 0101010100 0101010100 0101010100 0101000100 0101000101 0000000001',
  1),
 ('00001000100 0001110010 0001110010 0011101010 0011101010 0011101000 0011101000 0011000000 0011000000 0010000000 0010000000 0010000000',
  1),
 ('00000000010 0100010110 0100010101 0100010110 0101010100 0101000101 0110000101 0110100101 0110100101 0100100001 0100000101 0100000000',
  1),
 ('00010000001 1010010100 1010000101 1010000101 0010000100 0010001100 0010111100 0010110100 0010110100 0010000100 0000000000 0000000000'

# counts to IQ

In [197]:
from soft_info import get_repcode_IQ_map, get_KDEs

IQ_map = get_repcode_IQ_map(layout, ROUNDS)
print(IQ_map)

mumbai_calib_job = "cne879d72scg008df4j0" # 21.11.23
kde_dict, scaler_dict = get_KDEs(provider, tobecalib_backend = DEVICE)
print(kde_dict.keys())

{0: 1, 1: 3, 2: 8, 3: 14, 4: 19, 5: 25, 6: 23, 7: 18, 8: 12, 9: 7, 10: 1, 11: 3, 12: 8, 13: 14, 14: 19, 15: 25, 16: 23, 17: 18, 18: 12, 19: 7, 20: 1, 21: 3, 22: 8, 23: 14, 24: 19, 25: 25, 26: 23, 27: 18, 28: 12, 29: 7, 30: 1, 31: 3, 32: 8, 33: 14, 34: 19, 35: 25, 36: 23, 37: 18, 38: 12, 39: 7, 40: 1, 41: 3, 42: 8, 43: 14, 44: 19, 45: 25, 46: 23, 47: 18, 48: 12, 49: 7, 50: 1, 51: 3, 52: 8, 53: 14, 54: 19, 55: 25, 56: 23, 57: 18, 58: 12, 59: 7, 60: 1, 61: 3, 62: 8, 63: 14, 64: 19, 65: 25, 66: 23, 67: 18, 68: 12, 69: 7, 70: 1, 71: 3, 72: 8, 73: 14, 74: 19, 75: 25, 76: 23, 77: 18, 78: 12, 79: 7, 80: 1, 81: 3, 82: 8, 83: 14, 84: 19, 85: 25, 86: 23, 87: 18, 88: 12, 89: 7, 90: 1, 91: 3, 92: 8, 93: 14, 94: 19, 95: 25, 96: 23, 97: 18, 98: 12, 99: 7, 100: 1, 101: 3, 102: 8, 103: 14, 104: 19, 105: 25, 106: 23, 107: 18, 108: 12, 109: 7, 110: 0, 111: 2, 112: 5, 113: 11, 114: 16, 115: 22, 116: 24, 117: 21, 118: 15, 119: 10, 120: 4}
Found jobs for backend ibmq_mumbai with closest execution date 2023-

In [198]:
import numpy as np
from tqdm import tqdm

# Determine the total number of shots and number of qubits
total_shots = sum(shots for _, shots in sorted_counts)
len_IQ_array = len(IQ_map)
print("len_IQ_array: ", len_IQ_array)
print("len count str without spaces: ", len(sorted_counts[0][0].replace(" ", "")))

# Preallocate the NumPy array
IQ_memory2 = np.zeros((total_shots, len_IQ_array), dtype=np.complex128)

# Fill the array
shot_idx = 0
for count_str, shots in tqdm(sorted_counts):
    for _ in range(shots):
        num_spaces = 0
        inverted_count_str = count_str[::-1]   
        for IQ_idx, bit in enumerate(inverted_count_str):  # invert the order of count str to get IQ order
            if bit == ' ':
                num_spaces += 1
                continue
            cIQ_idx = IQ_idx - num_spaces
            qubit_idx = IQ_map[cIQ_idx]
            [kde0, kde1], scaler = kde_dict[qubit_idx], scaler_dict[qubit_idx]
            if bit == '0':
                sample = scaler.inverse_transform(kde0.sample(1, random_state=42))
                IQ_memory2[shot_idx, cIQ_idx] = complex(sample[0][0], sample[0][1])
            elif bit == '1':
                sample = scaler.inverse_transform(kde1.sample(1, random_state=42))
                IQ_memory2[shot_idx, cIQ_idx] = complex(sample[0][0], sample[0][1])
        shot_idx += 1

print("shape:", IQ_memory2.shape)
print("IQ_memory:", IQ_memory2)

len_IQ_array:  121
len count str without spaces:  121


  2%|▏         | 222/10000 [00:03<02:28, 65.98it/s]


KeyboardInterrupt: 

In [201]:
IQ_memory = np.zeros((total_shots, len_IQ_array), dtype=np.complex128)
# Initialize a dictionary to hold KDE sample requests
kde_samples_needed = {qubit_idx: {'0': 0, '1': 0} for qubit_idx in kde_dict.keys()}
sample_counters = {qubit_idx: {'0': 0, '1': 0} for qubit_idx in kde_dict.keys()}

# First pass: Aggregate how many samples we need from each KDE
for count_str, shots in tqdm(sorted_counts):
    num_spaces = 0
    inverted_count_str = count_str[::-1]
    for IQ_idx, bit in enumerate(inverted_count_str):
        if bit == ' ':
            num_spaces += 1
            continue
        qubit_idx = IQ_map[IQ_idx - num_spaces]
        kde_samples_needed[qubit_idx][bit] += shots

# print("kde_samples_needed:", kde_samples_needed)

# Perform KDE sampling and inverse transform
kde_samples = {}
for qubit_idx, needed_nb_samples in kde_samples_needed.items():
    [kde0, kde1], scaler = kde_dict[qubit_idx], scaler_dict[qubit_idx]
    if needed_nb_samples['0'] > 0:
        samples0 = scaler.inverse_transform(kde0.sample(needed_nb_samples['0'], random_state=42))
    else:
        samples0 = np.empty((0, 2)) 

    if needed_nb_samples['1'] > 0:
        samples1 = scaler.inverse_transform(kde1.sample(needed_nb_samples['1'], random_state=42))
    else:
        samples1 = np.empty((0, 2)) 

    kde_samples[qubit_idx] = {'0': samples0, '1': samples1}

# print("kde_samples:", kde_samples)


# Second pass: Assign the samples to IQ_memory
shot_idx = 0
for count_str, shots in tqdm(sorted_counts):
    for _ in range(shots):
        num_spaces = 0
        inverted_count_str = count_str[::-1]
        for IQ_idx, bit in enumerate(inverted_count_str):
            if bit == ' ':
                num_spaces += 1
                continue
            cIQ_idx = IQ_idx - num_spaces
            qubit_idx = IQ_map[cIQ_idx]
            sample_index = sample_counters[qubit_idx][bit]
            sample = kde_samples[qubit_idx][bit][sample_index]   
            # print(f"shot_idx: {shot_idx}, cIQ_idx: {cIQ_idx}, qubit_idx: {qubit_idx}, bit: {bit}, sample_index: {sample_index}") 
            # print("sample:", sample)     
            IQ_memory[shot_idx, cIQ_idx] = complex(sample[0], sample[1])
            sample_counters[qubit_idx][bit] += 1
            # print("IQ_memory2:", IQ_memory)
        shot_idx += 1

assert sample_counters == kde_samples_needed

print(IQ_memory.shape)
print("sample_counters:", sample_counters)

100%|██████████| 10000/10000 [00:00<00:00, 11823.07it/s]

(10000, 121)
sample_counters: {0: {'0': 8702, '1': 1298}, 1: {'0': 71723, '1': 38277}, 2: {'0': 8483, '1': 1517}, 3: {'0': 71564, '1': 38436}, 4: {'0': 8689, '1': 1311}, 5: {'0': 8484, '1': 1516}, 6: {'0': 0, '1': 0}, 7: {'0': 72537, '1': 37463}, 8: {'0': 71547, '1': 38453}, 9: {'0': 0, '1': 0}, 10: {'0': 8421, '1': 1579}, 11: {'0': 8488, '1': 1512}, 12: {'0': 71920, '1': 38080}, 13: {'0': 0, '1': 0}, 14: {'0': 71984, '1': 38016}, 15: {'0': 8441, '1': 1559}, 16: {'0': 8495, '1': 1505}, 17: {'0': 0, '1': 0}, 18: {'0': 71539, '1': 38461}, 19: {'0': 71789, '1': 38211}, 20: {'0': 0, '1': 0}, 21: {'0': 8492, '1': 1508}, 22: {'0': 8464, '1': 1536}, 23: {'0': 72318, '1': 37682}, 24: {'0': 8440, '1': 1560}, 25: {'0': 72024, '1': 37976}, 26: {'0': 0, '1': 0}}


In [202]:
print(IQ_memory)
print()
print(IQ_memory2)

# they won't be the same because I am sampling a batch with seed, and each of those will not be the same as sampling just one point with seed


[[-4.70970268e+07-2.03049795e+08j -2.18250298e+08-6.04834446e+08j
  -1.28773545e+08-3.47929586e+08j ... -4.55005451e+07-1.05476857e+08j
  -2.12231260e+07-1.08333817e+08j -8.55422155e+07-3.34507507e+08j]
 [-4.99384695e+07-2.01099515e+08j -1.83566319e+08-5.91110283e+08j
  -4.63293102e+07-3.94147294e+08j ... -5.91702895e+07-1.03306390e+08j
   2.52895409e+07-1.12934930e+08j -7.96229504e+07-2.96481390e+08j]
 [-2.77141621e+07-2.21224422e+08j -1.54771308e+08-6.25263049e+08j
  -7.56386793e+07-3.43935802e+08j ... -3.70107553e+07-9.19243721e+07j
  -3.93507964e+07-1.02409143e+08j -7.24626263e+07-2.46427870e+08j]
 ...
 [-4.79707598e+07-2.27661306e+08j -1.97292376e+08-6.15204749e+08j
  -1.24678984e+08-3.50703858e+08j ... -2.81074525e+07-1.21218386e+08j
  -5.76538446e+06-1.02802435e+08j -1.13246633e+08-2.88365960e+08j]
 [-5.65556509e+07-2.13919221e+08j -2.67081504e+08-5.89937533e+08j
  -4.90334746e+07-3.96371145e+08j ... -4.60606193e+07-1.37179278e+08j
  -2.04591917e+07-1.01883572e+08j -1.13904396e+

In [204]:
import numpy as np
print(np.array(IQ_memory).shape)

(10000, 121)


# Decode

In [205]:
from Scratch import create_or_load_kde_grid

grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, mumbai_calib_job, 300, 2, other_date=None)

Specified job execution date: 2023-11-21 19:54:05.610765+01:00
Found jobs for backend ibmq_mumbai with closest execution date 2023-11-21 18:54:05.610765+00:00.
Searching for ibmq_mumbai and 23.11.21_10h17_300pts_2std


In [206]:
import pymatching
import stim


circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=DISTANCE,
                                rounds=ROUNDS,
                                after_clifford_depolarization=p2Q, #two-qubit-fidelity,
                                after_reset_flip_probability=pRE, #reset error,
                                before_measure_flip_probability=pRO, #measurement error,
                                before_round_data_depolarization=pZ+pXY) #idle error)
# print(circuit)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)

In [256]:
import cpp_soft_info

matching = pymatching.Matching.from_detector_error_model(model)
# cpp_soft_info.reweight_edges_based_on_error_probs(matching._matching_graph, counts, False, "spitz")

p_data = 6.869e-1 # mean sherbrooke noise
num_errors = cpp_soft_info.decode_IQ_shots(matching._matching_graph, IQ_memory,
                                           ROUNDS, int(LOGICAL), IQ_map, grid_dict,
                                           processed_scaler_dict, p_data=-1, p_mixed=-1, #p_mixed=1e-80, for d=30
                                           common_measure=-1, _bimodal=False, merge_strategy = "independent")
print("num_errors:", num_errors, "out of", len(IQ_memory), "shots")

num_errors: 34 out of 10000 shots


In [208]:
matching = pymatching.Matching.from_detector_error_model(model)
num_errors = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, IQ_memory,
                                           ROUNDS, int(LOGICAL), IQ_map, grid_dict,
                                           processed_scaler_dict)
print("num_errors:", num_errors, "out of", len(IQ_memory), "shots")


num_errors: 563 out of 10000 shots


In [257]:
matching = pymatching.Matching.from_detector_error_model(model)

def weight_to_prob(weight):
    return 1/(1+np.exp(weight))

p_data = 1.869e-3# mean sherbrooke ECR error
p_mixed = p_data# Same as weighted
# p_meas = 1e-3
p_meas = 15.900e-2 # random found number


p_data = -1
p_mixed = -1
# p_meas = -1

num_errors = cpp_soft_info.decode_IQ_shots_flat_informed(matching._matching_graph, IQ_memory, 
                                           ROUNDS, int(LOGICAL), IQ_map, grid_dict, processed_scaler_dict,
                                           p_data = p_data, p_mixed = p_mixed, p_meas = p_meas, common_measure=-1)

print("num_errors:", num_errors, "out of", len(IQ_memory), "shots")
         
# takes 1s

num_errors: 19 out of 10000 shots
